In [1]:
#Loading the data

with open("11-0.txt", "r", encoding="utf-8") as file:
    text = file.read()

<h1>Part 1, 2</h1>

In [2]:
import numpy as np
import pandas as pd
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
# import all the resources for Natural Language Processing with Python
nltk.download("book")
nltk.download('punkt')


# Download NLTK resources if not already installed
nltk.download("stopwords")
nltk.download("wordnet")

# Load the text
with open("11-0.txt", "r", encoding="utf-8") as file:
    text = file.read()

# Convert to lowercase
text = text.lower()

# Remove numbers and non-alphabetic characters
text = re.sub(r'[^a-z\s]', '', text)

# Tokenize the text
tokens = nltk.word_tokenize(text)

# Remove stop words
stop_words = set(stopwords.words("english"))
filtered_tokens = [word for word in tokens if word not in stop_words]

# Lemmatization
lemmatizer = WordNetLemmatizer()
lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]

[nltk_data] Downloading collection 'book'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\abdur\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\abdur\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package chat80 to
[nltk_data]    |     C:\Users\abdur\AppData\Roaming\nltk_data...
[nltk_data]    |   Package chat80 is already up-to-date!
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\abdur\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package conll2000 to
[nltk_data]    |     C:\Users\abdur\AppData\Roaming\nltk_data...
[nltk_data]    |   Package conll2000 is already up-to-date!
[nltk_data]    | Downloading package conll2002 to
[nltk_data]    |     C:\Users\abdur\AppData\R

Cleaning the data

In [5]:
# split into words
tokens = word_tokenize(text)
# convert to lower case
tokens = [w.lower() for w in tokens]
# remove punctuation from each word
table = str.maketrans('', '', string.punctuation)
stripped = [w.translate(table) for w in tokens]
# remove remaining tokens that are not alphabetic
words = [word for word in stripped if word.isalpha()]
# filter out stop words
stop_words = set(stopwords.words('english'))
words = [w for w in words if not w in stop_words]
# lemmatizarion
wn = nltk.WordNetLemmatizer()
lemmas = [wn.lemmatize(word) for word in words] 

<h1>Part 3</h1>
10 most important words

In [6]:
occurences =[i for i,x in enumerate(lemmas) if x=='chapter']
occurences.append(len(lemmas))  # the last word (index) in the 'lemmas' variable is the end of the last chapter 

In [7]:
indices=[]
#First 12 occurence of the word 'chapter' are just the outline of the story, thus we delete them
chapter_points=occurences[12:] 
# building a list to contain for each chapter the indices for its start and ending.
for i in range (len(chapter_points)-1):
  indices.append((chapter_points[i],chapter_points[i+1]))
indices

[(134, 1113),
 (1113, 2115),
 (2115, 2938),
 (2938, 4162),
 (4162, 5200),
 (5200, 6421),
 (6421, 7542),
 (7542, 8720),
 (8720, 9837),
 (9837, 10851),
 (10851, 11752),
 (11752, 14371)]

In [8]:
# 'chapters' variable contains lists of lemmas for each chapter individually 
chapters = [lemmas [s:e] for s,e in indices]

# delete the word 'Alice' from each paragraph (avoiding getting 'Alice' as one of the most important words)
chapters_without_alice = [[word for word in chapter if word != 'alice'] for chapter in chapters]

# for each chapter, join the tokens together 
chapters_paragraphs=[' '.join(i) for i in chapters_without_alice]

In [11]:
# Calculating TF-IDF and finding top 10 words for each document
from sklearn.feature_extraction.text import TfidfVectorizer

# Define the model
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(chapters_paragraphs).toarray()
vocab = tfidf.vocabulary_
reverse_vocab = {v: k for k, v in vocab.items()}
feature_names = tfidf.get_feature_names_out()  # Use get_feature_names_out() to access feature names
df_tfidf = pd.DataFrame(X_tfidf, columns=feature_names)

idx = X_tfidf.argsort(axis=1)  # Sorting
tfidf_max10 = idx[:, -10:]  # Top 10
df_tfidf['top10'] = [[reverse_vocab.get(item) for item in row] for row in tfidf_max10]


In [13]:
#Printing Top 10 words for each chapter
for (i, item) in enumerate(df_tfidf['top10'] , start=1):
    print('Chapter', i, item)

Chapter 1 ['bottle', 'see', 'like', 'way', 'think', 'eat', 'key', 'door', 'bat', 'little']
Chapter 2 ['mabel', 'foot', 'said', 'dear', 'cat', 'swam', 'im', 'little', 'pool', 'mouse']
Chapter 3 ['soon', 'bird', 'know', 'thimble', 'dry', 'lory', 'prize', 'dodo', 'said', 'mouse']
Chapter 4 ['one', 'said', 'fan', 'bottle', 'glove', 'puppy', 'rabbit', 'window', 'little', 'bill']
Chapter 5 ['little', 'father', 'size', 'youth', 'egg', 'im', 'pigeon', 'serpent', 'said', 'caterpillar']
Chapter 6 ['cook', 'like', 'wow', 'duchess', 'pig', 'mad', 'baby', 'footman', 'cat', 'said']
Chapter 7 ['know', 'draw', 'tea', 'time', 'twinkle', 'hare', 'march', 'said', 'dormouse', 'hatter']
Chapter 8 ['executioner', 'procession', 'five', 'cat', 'soldier', 'gardener', 'king', 'hedgehog', 'said', 'queen']
Chapter 9 ['day', 'say', 'went', 'queen', 'moral', 'duchess', 'gryphon', 'mock', 'said', 'turtle']
Chapter 10 ['whiting', 'join', 'beautiful', 'soup', 'dance', 'lobster', 'said', 'gryphon', 'mock', 'turtle']
Ch

<h1>Part 4</h1>
Top 10 most used verbs with Alice

In [15]:
# splitting the story into sentences
sentences = nltk.sent_tokenize(text)

# Keep only the sentences that contains the word (alice) in order to find the verbs in these sentences as required.
sentences_with_alice = [sen for sen in sentences if 'alice' in sen.lower()]

In [16]:
# Tagging the sentences
sentences_tagged=[] # this list will contain each sentence tagged
for i in sentences_with_alice: 
  # Tokenizing each sentence individually
  wordsList = nltk.word_tokenize(i) 
  # Removing punctuation from each word
  stripped_2 = [ w.translate(table) for w in wordsList]
  # Removing stop words and non alphabtecis words 
  wordsList = [w.translate(table) for w in stripped_2 if not w in stop_words and w.isalpha()]
  # Using a Tagger. Which is part-of-speech tagger or POS-tagger. 
  tagged = nltk.pos_tag(wordsList) 
  sentences_tagged.append(tagged) 

In [17]:
# Finding the verbs in the tagged sentences
# All the tags that are related to verb: VB, VBD, VBG, VBN, VBP, VBZ
verb_tags=['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
verbs =[]
# keeping only the verbs 
for sen in sentences_tagged:
  verbs.append([verb for verb,tag in sen if tag in verb_tags])
all_verbs=[j for i in verbs for j in i] # This variable contains all the verbs in one list
#Finding the lemma for each verb in order to find the most frequent verb
verbs_lemmas = [wn.lemmatize(word,'v') for word in all_verbs] 

In [18]:
 # Creating a dataframe out of the list of verbs_lemmas
df_all_words = pd.DataFrame(verbs_lemmas, columns=['verb'])
# Grouping the verbs and find each verb frequency
df_all_words['counts'] = df_all_words.groupby(['verb'])['verb'].transform('count')
# Sorting the verbs by their frequencies
df_all_words = df_all_words.sort_values(by=['counts', 'verb'], ascending=[False, True]).reset_index()
# Finding most 10 frequent verbs
df_words = df_all_words.groupby('verb').first().sort_values(by='counts', ascending=False).reset_index()
print("Most 10 frequent verbs:")
print(df_words.head(10))
     

Most 10 frequent verbs:
    verb  index  counts
0    say     40     528
1     go     53     177
2    get     28      97
3  think     39      89
4  begin     27      85
5   look     46      81
6   come     65      81
7   know    107      80
8   make     34      74
9    see     49      65


Based on the analysis of the text, we have identified the ten most frequent verbs in the document. These verbs and their respective counts are as follows:

"say" with 528 occurrences.
"go" with 177 occurrences.
"get" with 97 occurrences.
"think" with 89 occurrences.
"begin" with 85 occurrences.
"look" with 81 occurrences.
"come" with 81 occurrences.
"know" with 80 occurrences.
"make" with 74 occurrences.
"see" with 65 occurrences.
These results provide insights into the verbs that are most frequently used in the text. Verbs like "say," "go," and "get" are the most common, indicating their significant presence in the narrative. Additionally, verbs like "think," "begin," and "look" are also frequently used, contributing to the overall understanding of the actions and events described in the text.

It's important to note that this analysis can help readers and researchers gain a better understanding of the text's themes, character actions, and storytelling style. The frequency of verbs provides valuable information about the narrative's dynamics and key elements.

In conclusion, this analysis of the most frequent verbs in the text sheds light on the language and narrative style employed by the author, contributing to a deeper comprehension of the text's content and themes.